In [2]:
import pandas as pd

df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\2.csv')
df

,Unnamed: 0,filename,record_num,clean_line1,clean_line2,line3,city,zip_code,label,phone,...,p_identifier,identifier2,identifier3,address_for_geocoding,latitude,longitude,precision_level,match_type,google_formatted_address,geocoding_status
0,23202,TF2016_260_275-0-ocr.csv,12,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 1-80 Exit 162 ( UT 280 ),<U+25A1> <U+2610> <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,...,NaN,84017_435-336-4421,84017_435-336-4421_NA,"Holiday Hills ( 66 ), 1-80 Exit 162 ( UT 280 )...",40.910923,-111.404793,ROOFTOP,PARTIAL,"118 S 500 W, Coalville, UT 84017, USA",SUCCESS
1,23208,TF2016_260_275-0-ocr.csv,23,Heber City ( 84032 ) Silver Eagle Country Store,435-654-4642 1590 US 40 S ( 1/2 mi S of jct US...,<U+2610> D,Heber City,84032,Silver Eagle Country Store,435-654-4642,...,NaN,84032_435-654-4642,84032_435-654-4642_NA,"Silver Eagle Country Store, 1590 US 40 S ( 1/2...",40.487803,-111.408741,ROOFTOP,PARTIAL,"1590 U.S. Hwy 40, Heber City, UT 84032, USA",SUCCESS
2,23214,TF2016_260_275-1-ocr.csv,23,Salt Lake City ( N ) ( 84054 ) Flying J Fuel S...,801-936-1408 1-215 Exit 27 ( UT 68 N ) FLYING,FLYING J 24,Salt Lake City,84054,Flying J Fuel Stop # 772,801-936-1408,...,NaN,84054_801-936-1408,84054_801-936-1408_Flying J,"Flying J, Flying J Fuel Stop # 772, 1-215 Exit...",40.843181,-111.931051,GEOMETRIC_CENTER,PARTIAL,"UT-68, North Salt Lake, UT 84054, USA",SUCCESS
3,23220,TF2016_260_275-1-ocr.csv,28,D Silver Creek ( 84060 ) Bell's Silver Creek J...,435-649-9243 1-80 Exit 146 ( US 40 ),24 S <U+2610>,Silver Creek,84060,Bell's Silver Creek Junction ( Sinclair ),435-649-9243,...,NaN,84060_435-649-9243,84060_435-649-9243_Sinclair,"Sinclair, Bell's Silver Creek Junction ( Sincl...",40.242212,-111.181999,RANGE_INTERPOLATED,PARTIAL,"146 U.S. Hwy 40, Utah, USA",SUCCESS
4,23226,TF2016_260_275-1-ocr.csv,14,Roosevelt ( 84066 ) Top Stop # 66 ( Chevron ),6 435-722-9940 1128 US 40 ( 1 mi W of town ),<U+2610> SO <U+25A1>,Roosevelt,84066,Top Stop # 66 ( Chevron ),435-722-9940,...,NaN,84066_435-722-9940,84066_435-722-9940_Chevron,"Chevron, Top Stop # 66 ( Chevron ), 1128 US 40...",40.455337,-109.543801,ROOFTOP,PARTIAL,"850 U.S. Hwy 40, Vernal, UT 84078, USA",SUCCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,37893,TF2016_020_035-2-ocr.csv,48,Dunnigan ( 95937 ) United Truck Stop,530-724-3477 1-5 Exit 554 ( CR 8 W ),24 <U+25A1><U+25A1> M,Dunnigan,95937,United Truck Stop,530-724-3477,...,NaN,95937_530-724-3477,95937_530-724-3477_NA,"United Truck Stop, 1-5 Exit 554 ( CR 8 W ), Du...",38.885238,-121.969580,APPROXIMATE,PARTIAL,"Dunnigan, CA, USA",SUCCESS
386,37895,TF2016_020_035-5-ocr.csv,27,Yuba City ( 95991 ) Dhami's Truck Stop S ( of ...,530-755-4564 CA 99 & Obanion ( 10 mi S ),M <U+25A1>,Yuba City,95991,Dhami's Truck Stop S ( of Shell town ) ),530-755-4564,...,NaN,95991_530-755-4564,95991_530-755-4564_Shell,"Shell, Dhami's Truck Stop S ( of Shell town ) ...",39.025758,-121.634276,ROOFTOP,PARTIAL,"1281 Obanion Rd, Yuba City, CA 95991, USA",SUCCESS
387,37901,TF2016_020_035-4-ocr.csv,30,B Redding ( 96003 ) KC's Corner Mart ( Chevron...,2 530-275-4375 1-5 Exit 687 ( Old Oregon Trail ),<U+2610> <U+25A1>,Redding,96003,KC's Corner Mart ( Chevron / CFN ),530-275-4375,...,NaN,96003_530-275-4375,96003_530-275-4375_Chevron,"Chevron, KC's Corner Mart ( Chevron / CFN ), 1...",40.605229,-122.313931,RANGE_INTERPOLATED,PARTIAL,"687 Old Oregon Trail, Redding, CA 96003, USA",SUCCESS
388,37906,TF2016_020_035-2-ocr.csv,37,Corning ( 96021 ) Rolling Hills Chevron,530-824-4990 1-5 Exit 628 ( Liberal Ave W ),MO,Corning,96021,Rolling Hills Chevron,530-824-4990,...,NaN,96021_530-824-4990,96021_530-824-4990_Chevron,"Chevron, Rolling Hills Chevron, 1-5 Exit 628 (...",39.874138,-122.202518,ROOFTOP,PARTIAL,"2655 Everett Freeman Way, Corning, CA 96021, USA",SUCCESS


In [3]:
# Compare address_for_geocoding and google_formatted_address columns
import numpy as np
from difflib import SequenceMatcher

# Basic comparison - check if they are exactly the same
df['addresses_match'] = df['address_for_geocoding'] == df['google_formatted_address']

# Calculate similarity ratio using difflib
def similarity_ratio(str1, str2):
    if pd.isna(str1) or pd.isna(str2):
        return 0.0
    return SequenceMatcher(None, str(str1).lower(), str(str2).lower()).ratio()

df['similarity_score'] = df.apply(lambda row: similarity_ratio(row['address_for_geocoding'], 
                                                               row['google_formatted_address']), axis=1)

# Display basic statistics
print("Address Comparison Statistics:")
print(f"Total records: {len(df)}")
print(f"Exact matches: {df['addresses_match'].sum()} ({df['addresses_match'].mean()*100:.1f}%)")
print(f"Average similarity score: {df['similarity_score'].mean():.3f}")
print(f"Median similarity score: {df['similarity_score'].median():.3f}")
print(f"Min similarity score: {df['similarity_score'].min():.3f}")
print(f"Max similarity score: {df['similarity_score'].max():.3f}")

Address Comparison Statistics:
Total records: 390
Exact matches: 0 (0.0%)
Average similarity score: 0.438
Median similarity score: 0.444
Min similarity score: 0.107
Max similarity score: 0.836


In [4]:
# Detailed comparison analysis
print("\n" + "="*50)
print("DETAILED COMPARISON ANALYSIS")
print("="*50)

# Group by similarity score ranges
print("\nSimilarity Score Distribution:")
similarity_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
similarity_labels = ['0-20%', '21-40%', '41-60%', '61-80%', '81-100%']
df['similarity_category'] = pd.cut(df['similarity_score'], bins=similarity_bins, labels=similarity_labels, include_lowest=True)
print(df['similarity_category'].value_counts().sort_index())

# Show examples of different similarity levels
print("\n" + "-"*50)
print("EXAMPLES BY SIMILARITY LEVEL")
print("-"*50)

for category in similarity_labels:
    examples = df[df['similarity_category'] == category].head(2)
    if len(examples) > 0:
        print(f"\n{category} Similarity Examples:")
        for idx, row in examples.iterrows():
            print(f"  Original: {row['address_for_geocoding'][:80]}...")
            print(f"  Google:   {row['google_formatted_address'][:80]}...")
            print(f"  Score:    {row['similarity_score']:.3f}")
            print()


DETAILED COMPARISON ANALYSIS

Similarity Score Distribution:
similarity_category
0-20%        9
21-40%     127
41-60%     234
61-80%      19
81-100%      1
Name: count, dtype: int64

--------------------------------------------------
EXAMPLES BY SIMILARITY LEVEL
--------------------------------------------------

0-20% Similarity Examples:
  Original: Texaco, Hallum Stores ( Texaco ), 1-40 Exit 48 ( US 93 N ), Kingman, AZ, 86401...
  Google:   US-93, United States...
  Score:    0.143

  Original: Shell, Gold Strike Auto Truck Plaza ( Shell ), I - 15 Exit 12, Jean, NV, 89019...
  Google:   I-15, United States...
  Score:    0.165


21-40% Similarity Examples:
  Original: Flying J, Flying J Fuel Stop # 772, 1-215 Exit 27 ( UT 68 N ) FLYING, Salt Lake ...
  Google:   UT-68, North Salt Lake, UT 84054, USA...
  Score:    0.364

  Original: Sinclair, Bell's Silver Creek Junction ( Sinclair ), 1-80 Exit 146 ( US 40 ), Si...
  Google:   146 U.S. Hwy 40, Utah, USA...
  Score:    0.220


41-60

In [5]:
# Analyze common differences and patterns
print("\n" + "="*50)
print("PATTERN ANALYSIS")
print("="*50)

# Look at records with low similarity but successful geocoding
low_similarity = df[(df['similarity_score'] < 0.3) & (df['geocoding_status'] == 'SUCCESS')]
print(f"\nRecords with low similarity (<30%) but successful geocoding: {len(low_similarity)}")

# Analyze by precision level
print("\nSimilarity scores by precision level:")
precision_analysis = df.groupby('precision_level')['similarity_score'].agg(['count', 'mean', 'std']).round(3)
print(precision_analysis)

# Analyze by match type
print("\nSimilarity scores by match type:")
match_analysis = df.groupby('match_type')['similarity_score'].agg(['count', 'mean', 'std']).round(3)
print(match_analysis)

# Show some examples where addresses are very different
print("\n" + "-"*50)
print("EXAMPLES OF VERY DIFFERENT ADDRESSES (Score < 0.2)")
print("-"*50)
very_different = df[df['similarity_score'] < 0.2].head(3)
for idx, row in very_different.iterrows():
    print(f"\nOriginal: {row['address_for_geocoding']}")
    print(f"Google:   {row['google_formatted_address']}")
    print(f"Score:    {row['similarity_score']:.3f}")
    print(f"Status:   {row['geocoding_status']} | Precision: {row['precision_level']} | Match: {row['match_type']}")
    print(f"Lat/Lng:  {row['latitude']:.6f}, {row['longitude']:.6f}")
    print("-" * 30)


PATTERN ANALYSIS

Records with low similarity (<30%) but successful geocoding: 41

Similarity scores by precision level:
                    count   mean    std
precision_level                        
APPROXIMATE            49  0.323  0.065
GEOMETRIC_CENTER       99  0.426  0.094
RANGE_INTERPOLATED     57  0.464  0.132
ROOFTOP               185  0.466  0.101

Similarity scores by match type:
            count   mean    std
match_type                     
EXACT           7  0.531  0.049
PARTIAL       383  0.436  0.111

--------------------------------------------------
EXAMPLES OF VERY DIFFERENT ADDRESSES (Score < 0.2)
--------------------------------------------------

Original: Texaco, Hallum Stores ( Texaco ), 1-40 Exit 48 ( US 93 N ), Kingman, AZ, 86401
Google:   US-93, United States
Score:    0.143
Status:   SUCCESS | Precision: GEOMETRIC_CENTER | Match: PARTIAL
Lat/Lng:  41.415801, -114.749709
------------------------------

Original: Shell, Gold Strike Auto Truck Plaza ( Shell )